# Loading Data

In [3]:
import pandas as pd

df = pd.read_json('results/PaperTab/2/2025-05-19-08-41.json')

In [45]:
import json
from dataclasses import dataclass

@dataclass
class ExperimentResult:
    user_chat: str
    response_chat: str

    user_prompt: str
    general_agent: str
    text_agent: str
    image_agent: str

    @staticmethod
    def from_json(user_ai_interaction: dict):
        user_chat = user_ai_interaction[0]['content'][0]['text']
        response_chat = user_ai_interaction[1]['content'][0]['text']
        
        # Extract agent responses
        user_prompt = ""
        general_agent = ""
        text_agent = ""
        image_agent = ""
        
        if "General Agent:" in user_chat:
            parts = user_chat.split("General Agent:")
            if len(parts) > 1:
                user_prompt = parts[0].strip()
                general_part = parts[1].split("Text Agent:")[0].strip()
                general_agent = general_part
                
        if "Text Agent:" in user_chat:
            parts = user_chat.split("Text Agent:")
            if len(parts) > 1:
                text_part = parts[1].split("Image Agent:")[0].strip()
                text_agent = text_part
                
        if "Image Agent:" in user_chat:
            parts = user_chat.split("Image Agent:")
            if len(parts) > 1:
                image_part = parts[1].strip()
                image_agent = image_part
        
        return ExperimentResult(
            user_chat=user_chat,
            user_prompt=user_prompt,
            general_agent=general_agent,
            text_agent=text_agent,
            image_agent=image_agent,
            response_chat=response_chat
        )

    def __str__(self):
        return \
            f"{'='*20} User Prompt: \n{self.user_prompt}\n" \
            f"{'='*20} General Agent: \n{self.general_agent}\n" \
            f"{'='*20} Text Agent: \n{self.text_agent}\n" \
            f"{'='*20} Image Agent: \n{self.image_agent}\n" \
            f"{'='*20} Response Chat: \n{self.response_chat}"

sample_result = ExperimentResult.from_json(df['ans_2_message'].iloc[8])
print(sample_result)

==================== User Prompt: 
You are tasked with summarizing and evaluating the collective responses provided by multiple agents. You have access to the following information:
Answers: The individual answers from all agents.
Using this information, perform the following tasks:
Analyze: Evaluate the quality, consistency, and relevance of each answer. Identify commonalities, discrepancies, or gaps in reasoning.
Synthesize: Summarize the most accurate and reliable information based on the evidence provided by the agents and their discussions.
Conclude: Provide a final, well-reasoned answer to the question or task. Your conclusion should reflect the consensus (if one exists) or the most credible and well-supported answer.
Based on the provided answers from all agents, summarize the final decision clearly. You should only return the final answer in this dictionary format: {"Answer": <Your final answer here>}. Don't give other information.
==================== General Agent: 
Based on 